# Multi-Class Prediction of Insulin Dosage Recommendation Evaluated from Patient Health Data

In [1]:
# importng the needed packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import xgboost

### Versions of the packages used :
listed to create a local env

In [2]:
np.__version__

'2.3.4'

In [3]:
pd.__version__

'2.3.3'

In [4]:
matplotlib.__version__

'3.10.7'

In [5]:
sns.__version__

'0.13.2'

In [6]:
sklearn.__version__

'1.7.2'

In [7]:
xgboost.__version__

'3.1.1'